In [1]:
import pyspark
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd
import pyarrow as pa

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [58]:

# import required module
import os
# assign directory
directory = 'green'
 
# iterate over files in
# that directory 
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f'Processing {filename}')
        
        df_temp = pd.read_parquet(f)
        
        df_temp.rename(columns = {'Airport_fee':'airport_fee'}, inplace = True)
        if directory == 'yellow':
            df_temp = df_temp.astype({"VendorID":'Int64', 
                                      "tpep_pickup_datetime":'datetime64',
                                      "tpep_dropoff_datetime":'datetime64',
                                      "passenger_count":'Int64',
                                      "trip_distance":'float64',
                                      "RatecodeID":'Int64',
                                      "store_and_fwd_flag":'str',
                                      "PULocationID":'Int64',
                                      "DOLocationID":'Int64',
                                      "payment_type":'Int64',
                                      "fare_amount":'float64',
                                      "extra":'float64',
                                      "mta_tax":'float64',
                                      "tip_amount":'float64',
                                      "tolls_amount":'float64',
                                      "improvement_surcharge":'float64',
                                      "total_amount":'float64',
                                      "congestion_surcharge":'float64',
                                      "airport_fee":'float64'
                                     }) 
        if directory == 'green':                             
            df_temp = df_temp.astype({"VendorID":'Int64', 
                                      "lpep_pickup_datetime":'datetime64',
                                      "lpep_dropoff_datetime":'datetime64',
                                      "store_and_fwd_flag":'str',
                                      "RatecodeID":'Int64',
                                      "PULocationID":'Int64',
                                      "DOLocationID":'Int64',
                                      "passenger_count":'Int64',
                                      "trip_distance":'float64',
                                      "fare_amount":'float64',
                                      "extra":'float64',
                                      "mta_tax":'float64',
                                      "tip_amount":'float64',
                                      "tolls_amount":'float64',
                                      "ehail_fee":'float64',
                                      "improvement_surcharge":'float64',
                                      "total_amount":'float64',
                                      "payment_type":'Int64',                                      
                                      "trip_type":'Int64',
                                      "congestion_surcharge":'float64'
                                     }) 
        df_temp.to_parquet(os.path.join(directory, 'processed',filename))
        
        
        

Processing green_tripdata_2022-01
Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'],
      dtype='object')
Processing green_tripdata_2022-02
Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'],
      dtype='object')
Processing green_tripdata_2022-03
Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID

In [14]:
df_green = spark.read.parquet('green/processed')
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

df_yellow = spark.read.parquet('yellow/processed')
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [15]:
common_columns = []

yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_columns.append(col)
        
print(common_columns)

['VendorID', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'payment_type', 'congestion_surcharge']


In [16]:

df_green_sel = df_green \
    .select(common_columns) \
    .withColumn('service_type', F.lit('green'))

df_yellow_sel = df_yellow \
    .select(common_columns) \
    .withColumn('service_type', F.lit('yellow'))

In [17]:
df_green_sel.printSchema()
df_yellow_sel.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- service_type: string (nullable = false)

root
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullabl

In [18]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [19]:
df_trips_data.withColumn("PUlocationID",F.col("PUlocationID").cast(LongType()))

DataFrame[VendorID: bigint, pickup_datetime: timestamp_ntz, dropoff_datetime: timestamp_ntz, store_and_fwd_flag: string, RatecodeID: bigint, PUlocationID: bigint, DOLocationID: bigint, passenger_count: bigint, trip_distance: double, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, payment_type: bigint, congestion_surcharge: double, service_type: string]

In [20]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 1045466|
|      yellow|49040585|
+------------+--------+



In [21]:
df_trips_data.createOrReplaceTempView('trips_data')

In [22]:
spark.sql("""
SELECT
    service_type,
    count(1)
FROM
    trips_data
GROUP BY 
    service_type
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 1045466|
|      yellow|49040585|
+------------+--------+



In [23]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [25]:
#df_result.write.parquet('output/')

df_result.coalesce(1).write.parquet('output/', mode='overwrite') #in order to reduce amount of parquet files

In [26]:
df_test = spark.read.parquet('output').show(5)

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|revenue_monthly_congestion_surcharge|avg_montly_passenger_count|avg_montly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|          25|2022-

In [ ]:
# revenue report:

In [4]:

df_green2 = spark.read.parquet('green/processed')

df_green2.createOrReplaceTempView('green2')

df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green2
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

df_green_revenue \
    .repartition(20) \
    .write.parquet('output/revenue/green', mode='overwrite')

In [6]:
df_yellow2 = spark.read.parquet('yellow/processed')

df_yellow2.createOrReplaceTempView('yellow2')

df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow2
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

df_yellow_revenue \
    .repartition(20) \
    .write.parquet('output/revenue/yellow', mode='overwrite')

In [7]:
#joining 2 big tables - shuffiling:
df_green_revenue = spark.read.parquet('output/revenue/green')
df_yellow_revenue = spark.read.parquet('output/revenue/yellow')   

In [8]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [9]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [10]:
df_join.write.parquet('output/revenue/total', mode='overwrite')

In [11]:
#joining big table and small table - broadcasting:
df_join = spark.read.parquet('output/revenue/total')

In [12]:
df_join

DataFrame[hour: timestamp, zone: bigint, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [17]:
df_zones = spark.read.option("header",True).csv('‪taxi+_zone_lookup.csv')

In [18]:
df_zones.head(5)

[Row(LocationID='1', Borough='EWR', Zone='Newark Airport', service_zone='EWR'),
 Row(LocationID='2', Borough='Queens', Zone='Jamaica Bay', service_zone='Boro Zone'),
 Row(LocationID='3', Borough='Bronx', Zone='Allerton/Pelham Gardens', service_zone='Boro Zone'),
 Row(LocationID='4', Borough='Manhattan', Zone='Alphabet City', service_zone='Yellow Zone'),
 Row(LocationID='5', Borough='Staten Island', Zone='Arden Heights', service_zone='Boro Zone')]

In [19]:
df_result2 = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [22]:
df_result2.drop('LocationID', 'zone').write.parquet('output/revenue-zones')

In [26]:
spark.read.parquet('output/revenue-zones').show(5)

+-------------------+------------+--------------------+-------------+---------------------+---------+------------+
|               hour|green_amount|green_number_records|yellow_amount|yellow_number_records|  Borough|service_zone|
+-------------------+------------+--------------------+-------------+---------------------+---------+------------+
|2021-12-31 23:00:00|        null|                null|         14.1|                    2|Manhattan|   Boro Zone|
|2021-12-31 23:00:00|        null|                null|        79.37|                    1|   Queens|    Airports|
|2021-12-31 23:00:00|        null|                null|        11.76|                    1|Manhattan| Yellow Zone|
|2021-12-31 23:00:00|        null|                null|          6.3|                    1|Manhattan| Yellow Zone|
|2022-01-01 00:00:00|        43.0|                   5|        30.86|                    3|Manhattan|   Boro Zone|
+-------------------+------------+--------------------+-------------+-----------